CELL 1 — Install & Imports, set folders (edit paths if needed)

In [ ]:
# Cell 1 — install libs & set up folders
!pip install -q opencv-python-headless matplotlib tqdm

import os, glob, shutil
from pathlib import Path
import cv2
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
from google.colab import files

# ------------ Edit these paths if your files are in other locations ------------
# Put your TuSimple images in /content/tusimple_images
# Put corresponding binary GT masks (same basename) in /content/tusimple_masks
IMAGES_DIR = "/content/tusimple_images"
GT_MASKS_DIR = "/content/tusimple_masks"

# Output folder (we will save overlays, transparent PNGs, and predicted masks here)
OUT_DIR = "/content/model2_outputs"
os.makedirs(IMAGES_DIR, exist_ok=True)
os.makedirs(GT_MASKS_DIR, exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)

print("Images dir:", IMAGES_DIR)
print("GT masks dir:", GT_MASKS_DIR)
print("Outputs will be saved to:", OUT_DIR)


CELL 2 — Helper functions (preprocess, Otsu+Canny, Hough, averaging, draw)

In [ ]:
# Cell 2 — pipeline helper functions (clear and commented)
def preprocess_and_resize(img_bgr, resize_to=(960,540)):
    """Resize image and return (resized_color, blurred_gray)"""
    if img_bgr is None:
        return None, None
    resized = cv2.resize(img_bgr, resize_to, interpolation=cv2.INTER_LINEAR)
    gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5,5), 0)
    return resized, blurred

def otsu_then_canny(blur_img, canny1=50, canny2=150):
    """Apply Otsu thresholding then Canny edge detection"""
    if blur_img is None:
        return None, None
    _, otsu = cv2.threshold(blur_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    edges = cv2.Canny(otsu, canny1, canny2)
    return otsu, edges

def detect_hough(edges, rho=1, theta=np.pi/180, thresh=50, min_len=50, max_gap=20):
    """Detect line segments using Probabilistic Hough Transform"""
    if edges is None:
        return []
    lines = cv2.HoughLinesP(edges, rho, theta, thresh, minLineLength=min_len, maxLineGap=max_gap)
    if lines is None:
        return []
    return lines.reshape(-1,4)

def separate_lines_by_slope(lines, slope_thresh=0.3):
    """Separate detected line segments into left and right groups using slope sign"""
    left, right = [], []
    for x1,y1,x2,y2 in lines:
        if x2 == x1:
            continue
        slope = (y2 - y1) / (x2 - x1 + 1e-9)
        if abs(slope) < slope_thresh:
            continue
        if slope < 0:
            left.append((x1,y1,x2,y2))
        else:
            right.append((x1,y1,x2,y2))
    return left, right

def avg_slope_intercept(lines):
    """Compute average slope and intercept for a list of line segments"""
    if not lines:
        return None
    slopes, intercepts = [], []
    for x1,y1,x2,y2 in lines:
        if x2 == x1:
            continue
        s = (y2 - y1) / (x2 - x1 + 1e-9)
        b = y1 - s * x1
        slopes.append(s); intercepts.append(b)
    if not slopes:
        return None
    return float(np.mean(slopes)), float(np.mean(intercepts))

def line_points_from_slope_intercept(y_top, y_bottom, line):
    """Given slope/intercept, compute integer line endpoints (x_top, y_top, x_bottom, y_bottom)"""
    s, b = line
    if abs(s) < 1e-6:
        s = 1e-6
    x_top = int((y_top - b) / s)
    x_bottom = int((y_bottom - b) / s)
    return x_top, int(y_top), x_bottom, int(y_bottom)

def draw_average_lines(resized_img, left_line, right_line, color=(0,255,0), thickness=6):
    """Draw averaged left and right lines on image and return blended overlay (BGR)"""
    overlay = resized_img.copy()
    h, w = resized_img.shape[:2]
    y_bottom = h
    y_top = int(h * 0.6)
    if left_line is not None:
        x1,y1,x2,y2 = line_points_from_slope_intercept(y_top, y_bottom, left_line)
        cv2.line(overlay, (x1,y1), (x2,y2), color, thickness)
    if right_line is not None:
        x1,y1,x2,y2 = line_points_from_slope_intercept(y_top, y_bottom, right_line)
        cv2.line(overlay, (x1,y1), (x2,y2), color, thickness)
    blended = cv2.addWeighted(resized_img, 0.8, overlay, 0.6, 0)
    return blended


CELL 3 — Transparent overlay & mask creator

In [ ]:
# Cell 3 — create transparent RGBA overlay (lines only) and binary mask
def create_transparent_and_mask(resized_img, left_line, right_line, line_color=(0,255,0), thickness=6):
    """
    Returns:
      - bgra: BGRA image with lane lines drawn and alpha where lines exist
      - mask: single-channel binary mask (255 where lane lines exist)
    """
    h, w = resized_img.shape[:2]
    blank = np.zeros((h, w, 3), dtype=np.uint8)
    y_bottom = h
    y_top = int(h * 0.6)
    if left_line is not None:
        x1,y1,x2,y2 = line_points_from_slope_intercept(y_top, y_bottom, left_line)
        cv2.line(blank, (x1,y1), (x2,y2), line_color, thickness)
    if right_line is not None:
        x1,y1,x2,y2 = line_points_from_slope_intercept(y_top, y_bottom, right_line)
        cv2.line(blank, (x1,y1), (x2,y2), line_color, thickness)
    # alpha channel: where any pixel non-zero
    alpha = (np.any(blank != 0, axis=2)).astype(np.uint8) * 255
    bgra = cv2.cvtColor(blank, cv2.COLOR_BGR2BGRA)
    bgra[..., 3] = alpha
    mask = alpha.copy()
    return bgra, mask


CELL 4 — Single-image pipeline returning intermediates

In [ ]:
# Cell 4 — pipeline for a single image (returns dict)
def process_single_image(image_path,
                         resize_to=(960,540),
                         canny1=50, canny2=150,
                         hough_rho=1, hough_theta=np.pi/180, hough_thresh=50, hough_minlen=50, hough_maxgap=20,
                         slope_thresh=0.3):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Could not read image: {image_path}")
    resized, blur = preprocess_and_resize(img, resize_to)
    otsu, edges = otsu_then_canny(blur, canny1, canny2)
    lines = detect_hough(edges, rho=hough_rho, theta=hough_theta, thresh=hough_thresh,
                         min_len=hough_minlen, max_gap=hough_maxgap)
    left_seg, right_seg = separate_lines_by_slope(lines, slope_thresh=slope_thresh)
    left_avg = avg_slope_intercept(left_seg)
    right_avg = avg_slope_intercept(right_seg)
    overlay = draw_average_lines(resized, left_avg, right_avg)
    trans_rgba, pred_mask = create_transparent_and_mask(resized, left_avg, right_avg)
    return {
        "resized": resized,
        "blur": blur,
        "otsu": otsu,
        "edges": edges,
        "lines": lines,
        "left_avg": left_avg,
        "right_avg": right_avg,
        "overlay": overlay,
        "transparent": trans_rgba,
        "pred_mask": pred_mask
    }


CELL 5 — IoU helper (compares predicted mask to GT mask)

In [ ]:
# Cell 5 — compute IoU between predicted binary mask and GT mask
def compute_iou(gt_mask, pred_mask, thresh=127):
    """
    gt_mask, pred_mask: single-channel images (uint8)
    returns IoU float
    """
    if gt_mask is None or pred_mask is None:
        return None
    gt_bin = (gt_mask > thresh).astype(np.uint8)
    pred_bin = (pred_mask > thresh).astype(np.uint8)
    inter = np.logical_and(gt_bin, pred_bin).sum()
    union = np.logical_or(gt_bin, pred_bin).sum()
    if union == 0:
        return 1.0 if inter == 0 else 0.0
    return inter / union


CELL 6 — Process the whole TuSimple folder sequentially, save outputs, display first N inline, compute IoU if GT exists

In [ ]:
# Cell 6 — batch processing over all images in IMAGES_DIR
def process_tusimple_folder(images_dir=IMAGES_DIR, gt_dir=GT_MASKS_DIR, out_dir=OUT_DIR,
                            resize_to=(960,540), canny1=50, canny2=150,
                            hough_rho=1, hough_theta=np.pi/180, hough_thresh=40, hough_minlen=30, hough_maxgap=40,
                            slope_thresh=0.3, max_show=35):
    image_paths = sorted([p for p in glob.glob(os.path.join(images_dir, "*")) if p.lower().endswith(('.jpg','.jpeg','.png'))])
    if not image_paths:
        print("No images found in", images_dir)
        return []
    results_summary = []
    for i, img_path in enumerate(tqdm(image_paths, desc="Processing TuSimple images")):
        base = Path(img_path).stem
        # process
        try:
            res = process_single_image(img_path,
                                       resize_to=resize_to,
                                       canny1=canny1, canny2=canny2,
                                       hough_rho=hough_rho, hough_theta=hough_theta,
                                       hough_thresh=hough_thresh, hough_minlen=hough_minlen,
                                       hough_maxgap=hough_maxgap, slope_thresh=slope_thresh)
        except Exception as e:
            print("Error processing", img_path, "->", e)
            continue
        # save outputs
        out_overlay = os.path.join(out_dir, base + "_overlay.jpg")
        out_trans   = os.path.join(out_dir, base + "_transparent.png")
        out_mask    = os.path.join(out_dir, base + "_pred_mask.png")
        cv2.imwrite(out_overlay, res["overlay"])
        cv2.imwrite(out_trans, res["transparent"])   # BGRA saved preserving alpha
        cv2.imwrite(out_mask, res["pred_mask"])
        # compute IoU if GT exists
        gt_path = os.path.join(gt_dir, base + ".png")  # assume GT mask has .png extension and same basename
        iou = None
        if os.path.exists(gt_path):
            gt = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)
            if gt is not None:
                iou = compute_iou(gt, res["pred_mask"])
        results_summary.append({
            "image": img_path,
            "overlay": out_overlay,
            "transparent": out_trans,
            "pred_mask": out_mask,
            "gt_path": gt_path if os.path.exists(gt_path) else None,
            "iou": iou
        })
        # display first few results inline
        if i < max_show:
            plt.figure(figsize=(15,5))
            plt.subplot(1,4,1); plt.title("Original (resized)"); plt.imshow(cv2.cvtColor(res["resized"], cv2.COLOR_BGR2RGB)); plt.axis('off')
            plt.subplot(1,4,2); plt.title("Edges"); plt.imshow(res["edges"], cmap='gray'); plt.axis('off')
            plt.subplot(1,4,3); plt.title("Overlay"); plt.imshow(cv2.cvtColor(res["overlay"], cv2.COLOR_BGR2RGB)); plt.axis('off')
            plt.subplot(1,4,4); plt.title("Pred Mask"); plt.imshow(res["pred_mask"], cmap='gray'); plt.axis('off')
            plt.show()
            if iou is not None:
                print(f"IoU with GT ({base}.png): {iou:.4f}")
            else:
                print(f"GT not found for {base}")
    print(f"Processing finished. Outputs saved to: {out_dir}")
    return results_summary

# Run it (this will process all images found)
summary = process_tusimple_folder()

CELL 7 — Print summary table (first 20 entries) and basic stats

In [ ]:
# Cell 7 — show a brief summary
import pandas as pd
df = pd.DataFrame(summary)
if not df.empty:
    display(df.head(35))
    ious = [r['iou'] for r in summary if r['iou'] is not None]
    if ious:
        print("Mean IoU (computed only where GT mask present):", np.mean(ious))
    else:
        print("No IoU computed (GT masks not found or unreadable).")
else:
    print("No results to summarize.")


CELL 8 — Zip outputs for download

In [ ]:
# Cell 8 — zip the output folder for download
zip_path = "/content/model2_outputs.zip"
shutil.make_archive(zip_path.replace('.zip',''), 'zip', OUT_DIR)
print("Zipped outputs to:", zip_path)
# Optionally download (uncomment to trigger browser download)
# files.download(zip_path)
